In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ultramedical-model-sft-3b/other/default/1/Model_SFT_3B/adapter_model.safetensors
/kaggle/input/ultramedical-model-sft-3b/other/default/1/Model_SFT_3B/training_args.bin
/kaggle/input/ultramedical-model-sft-3b/other/default/1/Model_SFT_3B/adapter_config.json
/kaggle/input/ultramedical-model-sft-3b/other/default/1/Model_SFT_3B/README.md
/kaggle/input/ultramedical-model-sft-3b/other/default/1/Model_SFT_3B/tokenizer.json
/kaggle/input/ultramedical-model-sft-3b/other/default/1/Model_SFT_3B/tokenizer_config.json
/kaggle/input/ultramedical-model-sft-3b/other/default/1/Model_SFT_3B/chat_template.jinja
/kaggle/input/ultramedical-model-sft-3b/other/default/1/Model_SFT_3B/special_tokens_map.json
/kaggle/input/reward-model-3b/other/default/1/Reward_Model_3B/adapter_model.safetensors
/kaggle/input/reward-model-3b/other/default/1/Reward_Model_3B/adapter_config.json
/kaggle/input/reward-model-3b/other/default/1/Reward_Model_3B/README.md
/kaggle/input/reward-model-3b/other/default/1/Rewar

## Setup and Installation

In [2]:
# Install required packages
!pip install -q transformers datasets accelerate peft bitsandbytes trl torch ijson huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 33.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.9/532.9 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.0/149.0 kB 7.9 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import login
login()

In [4]:
import ijson

with open("/kaggle/input/ultramedical-preference/ultramedical_preference.json", "rb") as f:
    first_item = next(ijson.items(f, "item"))
    print(first_item)


{'prompt_id': 'WikiInstruct,8304', 'label_type': 'length', 'prompt': 'Investigate the intricacies of immunometabolism, a distinct subfield of immunology that examines the interconnection between cellular metabolic processes and the functional attributes of immune cells. Clarify the mechanisms by which this symbiosis modulates the comprehensive immune response, and analyze its integration within the broader spectrum of immunological studies, with an emphasis on the implications for metabolic diseases and the consequential effects on the proficiency of the immune system.', 'chosen': [{'content': 'Investigate the intricacies of immunometabolism, a distinct subfield of immunology that examines the interconnection between cellular metabolic processes and the functional attributes of immune cells. Clarify the mechanisms by which this symbiosis modulates the comprehensive immune response, and analyze its integration within the broader spectrum of immunological studies, with an emphasis on the

In [5]:
# Import libraries
import torch
import json
import random
import pandas as pd
from datasets import Dataset, load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    AutoModelForSequenceClassification,
    BitsAndBytesConfig,
    TrainingArguments,
)
from peft import LoraConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training
from trl import (
    RewardTrainer,
    RewardConfig,
    PPOTrainer,
    PPOConfig,
    AutoModelForCausalLMWithValueHead
)
from transformers import DataCollatorWithPadding
import numpy as np
from typing import Dict, List

2026-01-30 07:57:26.781348: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769759847.115366      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769759847.211977      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1769759848.094653      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769759848.094697      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769759848.094700      55 computation_placer.cc:177] computation placer alr

In [6]:
print("✅ All packages imported successfully!")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

✅ All packages imported successfully!
PyTorch version: 2.8.0+cu126
CUDA available: True
GPU: Tesla P100-PCIE-16GB


## Configuration

In [7]:
# Model Configuration
BASE_MODEL = "meta-llama/Llama-3.2-3B-Instruct"
MODEL_SFT_PATH = "/kaggle/input/ultramedical-model-sft-3b/other/default/1/Model_SFT_3B"  
#REWARD_MODEL_PATH = "/kaggle/working/reward_model"
REWARD_MODEL_PATH = "/kaggle/input/reward-model-3b/other/default/1/Reward_Model_3B"
MODEL_C_PATH = "./kaggle/working/model_c_rlhf_ppo"


In [8]:
# Data Configuration
PREFERENCE_DATA_PATH = "/kaggle/input/ultramedical-preference/ultramedical_preference.json" 

In [9]:
# Training Configuration
REWARD_TRAINING_CONFIG = {
    "output_dir": REWARD_MODEL_PATH,
    "num_train_epochs": 1,
    "per_device_train_batch_size": 1,
    "gradient_accumulation_steps": 16,
    "learning_rate": 1e-5,
    "lr_scheduler_type": "cosine",
    "warmup_ratio": 0.1,
    "logging_steps": 10,
    "save_steps": 100,
    "eval_steps": 100,
    "save_total_limit": 2,
    "bf16": True,
    "gradient_checkpointing": True,
    "remove_unused_columns": False,
}

PPO_CONFIG = {
    "learning_rate": 1e-5,
    "batch_size": 16,
    "mini_batch_size": 1,
    "gradient_accumulation_steps": 16,
}

# LoRA Configuration
from peft import TaskType

LORA_CONFIG = {
    "r": 16,
    "lora_alpha": 32,
    "lora_dropout": 0.05,
    "bias": "none",
    "task_type": TaskType.SEQ_CLS,
    "target_modules": ["q_proj", "k_proj", "v_proj", "o_proj"]
}

# Quantization Configuration
BNB_CONFIG = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)


print("✅ Configuration set!")

✅ Configuration set!


## Load and Preprocess Preference Data

In [10]:
def load_preference_data(
    file_path: str,
    num_samples: int = 1000,
    seed: int = 42
) -> Dataset:
    """
    Load and preprocess preference data from JSON file
    
    Expected format:
    {
        "prompt": "...",
        "chosen": [{"role": "user", "content": "..."}, {"role": "assistant", "content": "..."}],
        "rejected": [{"role": "user", "content": "..."}, {"role": "assistant", "content": "..."}]
    }
    """

    print("Loading preference data...")

    # Load JSON
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    # Normalize structure
    if isinstance(data, dict):
        if "data" in data:
            data = data["data"]
        elif "rows" in data:
            data = data["rows"]

    print(f"Total available preference pairs: {len(data)}")

    # 🔹 Random sampling
    random.seed(seed)
    if len(data) > num_samples:
        data = random.sample(data, num_samples)
        print(f"Randomly sampled {num_samples} examples")
    else:
        print("Dataset smaller than requested sample size — using all data")

    processed_data = []

    for idx, example in enumerate(data):
        try:
            # Prompt
            prompt = example.get("prompt")
            if not prompt and "chosen" in example and isinstance(example["chosen"], list):
                prompt = example["chosen"][0].get("content", "")

            if not prompt:
                continue

            # Chosen
            if isinstance(example.get("chosen"), list):
                chosen = next(
                    (t["content"] for t in example["chosen"] if t.get("role") == "assistant"),
                    None
                )
            else:
                chosen = example.get("chosen")

            # Rejected
            if isinstance(example.get("rejected"), list):
                rejected = next(
                    (t["content"] for t in example["rejected"] if t.get("role") == "assistant"),
                    None
                )
            else:
                rejected = example.get("rejected")

            if chosen and rejected:
                processed_data.append({
                    "prompt": prompt,
                    "chosen": chosen,
                    "rejected": rejected
                })

        except Exception as e:
            print(f"⚠️ Skipped example {idx}: {e}")

    print(f"Successfully processed {len(processed_data)} examples")

    dataset = Dataset.from_pandas(pd.DataFrame(processed_data))

    # Preview
    print("\n📊 Sample:")
    print("Prompt:", dataset[0]["prompt"][:100], "...")
    print("Chosen:", dataset[0]["chosen"][:100], "...")
    print("Rejected:", dataset[0]["rejected"][:100], "...")

    return dataset


In [11]:
# Load the data
preference_dataset = load_preference_data(PREFERENCE_DATA_PATH)

Loading preference data...
Total available preference pairs: 109353
Randomly sampled 1000 examples
Successfully processed 1000 examples

📊 Sample:
Prompt: Ive been recently having more nose bleeds lately. 4-5 times per day to be exact. And I have the wors ...
Chosen: 

I'm so sorry to hear that you're experiencing frequent nosebleeds and a persistent cough. While it ...
Rejected: I'm not a doctor, but I can provide you with some general information. Frequent nosebleeds accompani ...


In [12]:
# Split into train/eval
split_dataset = preference_dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = split_dataset['train']
eval_dataset = split_dataset['test']

print(f"\n✅ Dataset prepared!")
print(f"   Training samples: {len(train_dataset)}")
print(f"   Evaluation samples: {len(eval_dataset)}")


✅ Dataset prepared!
   Training samples: 900
   Evaluation samples: 100


## Tokenization for Reward Model

In [13]:
print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
print("✅ Tokenizer loaded successfully")

Loading tokenizer...


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

✅ Tokenizer loaded successfully


In [14]:
def format_for_reward_model(examples: Dict) -> Dict:
    """
    Format examples for reward model training
    TRL RewardTrainer expects 'chosen' and 'rejected' as full text strings
    """
    
    prompts = examples['prompt']
    chosen = examples['chosen']
    rejected = examples['rejected']
    
    # Format in Llama chat format
    def format_text(prompt, response):
        text = f"<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\n{prompt}<|eot_id|>"
        text += f"<|start_header_id|>assistant<|end_header_id|>\n\n{response}<|eot_id|>"
        return text
    
    # Create full text for chosen and rejected
    chosen_texts = [format_text(p, c) for p, c in zip(prompts, chosen)]
    rejected_texts = [format_text(p, r) for p, r in zip(prompts, rejected)]
    
    return {
        'chosen': chosen_texts,
        'rejected': rejected_texts,
    }

# Apply formatting (keep as text, don't tokenize yet)
print("Formatting dataset for reward model...")
formatted_train = train_dataset.map(
    format_for_reward_model,
    batched=True,
    remove_columns=train_dataset.column_names
)

formatted_eval = eval_dataset.map(
    format_for_reward_model,
    batched=True,
    remove_columns=eval_dataset.column_names
)

print("✅ Formatting complete!")
print(f"Sample chosen: {formatted_train[0]['chosen'][:200]}...")
print(f"Sample rejected: {formatted_train[0]['rejected'][:200]}...")

Formatting dataset for reward model...


Map:   0%|          | 0/900 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

✅ Formatting complete!
Sample chosen: <|begin_of_text|><|start_header_id|>user<|end_header_id|>

A 25 yr old female presented with malaise and generalized weakness since 6 months. Her appetite is reduced and she has giddiness and palpitat...
Sample rejected: <|begin_of_text|><|start_header_id|>user<|end_header_id|>

A 25 yr old female presented with malaise and generalized weakness since 6 months. Her appetite is reduced and she has giddiness and palpitat...


In [15]:
# print("Loading tokenizer...")
# tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
# tokenizer.pad_token = tokenizer.eos_token
# tokenizer.padding_side = "right"

# def format_for_reward_model(examples: Dict) -> Dict:
#     """
#     Format examples for reward model training
    
#     Returns:
#         Dict with 'input_ids_chosen', 'attention_mask_chosen',
#         'input_ids_rejected', 'attention_mask_rejected'
#     """
    
#     prompts = examples['prompt']
#     chosen = examples['chosen']
#     rejected = examples['rejected']
    
#     # Format in Llama chat format
#     def format_text(prompt, response):
#         text = f"<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\n{prompt}<|eot_id|>"
#         text += f"<|start_header_id|>assistant<|end_header_id|>\n\n{response}<|eot_id|>"
#         return text
    
#     # Tokenize chosen
#     chosen_texts = [format_text(p, c) for p, c in zip(prompts, chosen)]
#     chosen_tokens = tokenizer(
#         chosen_texts,
#         truncation=True,
#         max_length=1024,
#         padding="max_length",
#         return_tensors="pt"
#     )
    
#     # Tokenize rejected
#     rejected_texts = [format_text(p, r) for p, r in zip(prompts, rejected)]
#     rejected_tokens = tokenizer(
#         rejected_texts,
#         truncation=True,
#         max_length=1024,
#         padding="max_length",
#         return_tensors="pt"
#     )
    
#     return {
#         'input_ids_chosen': chosen_tokens['input_ids'],
#         'attention_mask_chosen': chosen_tokens['attention_mask'],
#         'input_ids_rejected': rejected_tokens['input_ids'],
#         'attention_mask_rejected': rejected_tokens['attention_mask'],
#     }

# # Apply tokenization
# print("Tokenizing dataset for reward model...")
# tokenized_train = train_dataset.map(
#     format_for_reward_model,
#     batched=True,
#     remove_columns=train_dataset.column_names
# )

# tokenized_eval = eval_dataset.map(
#     format_for_reward_model,
#     batched=True,
#     remove_columns=eval_dataset.column_names
# )

# print("✅ Tokenization complete!")


## Train Reward Model

In [18]:
print("\n" + "="*80)
print("STEP 1: TRAINING REWARD MODEL")
print("="*80)

# Load base model for reward modeling
print("Loading base model for reward modeling...")
reward_base_model = AutoModelForSequenceClassification.from_pretrained(
    BASE_MODEL,
    num_labels=1,  # Regression for reward score
    quantization_config=BNB_CONFIG,
    device_map="auto",
    trust_remote_code=True,
)

# Prepare for training
reward_base_model = prepare_model_for_kbit_training(reward_base_model)

# Add LoRA
lora_config = LoraConfig(**LORA_CONFIG)
reward_model = get_peft_model(reward_base_model, lora_config)

# Print trainable parameters
trainable_params = sum(p.numel() for p in reward_model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in reward_model.parameters())
print(f"Trainable: {trainable_params:,} / {total_params:,} ({100*trainable_params/total_params:.2f}%)")

# Manual Training Loop (More Reliable)
print("\n🚀 Starting manual reward model training...")

from torch.utils.data import DataLoader
from tqdm import tqdm

# Create dataloader
train_dataloader = DataLoader(
    formatted_train,
    batch_size=REWARD_TRAINING_CONFIG["per_device_train_batch_size"],
    shuffle=True,
    drop_last=True
)

eval_dataloader = DataLoader(
    formatted_eval,
    batch_size=REWARD_TRAINING_CONFIG["per_device_train_batch_size"],
    shuffle=False,
    drop_last=True
)

# Optimizer and scheduler
from transformers import get_cosine_schedule_with_warmup

optimizer = torch.optim.AdamW(
    reward_model.parameters(),
    lr=REWARD_TRAINING_CONFIG["learning_rate"]
)

num_training_steps = len(train_dataloader) * REWARD_TRAINING_CONFIG["num_train_epochs"]
num_warmup_steps = int(num_training_steps * REWARD_TRAINING_CONFIG["warmup_ratio"])

scheduler = get_cosine_schedule_with_warmup(
    optimizer,
    num_warmup_steps=num_warmup_steps,
    num_training_steps=num_training_steps
)

# Training loop
reward_model.train()
device = reward_model.device if hasattr(reward_model, 'device') else next(reward_model.parameters()).device
print(f"Training on device: {device}")

gradient_accumulation_steps = REWARD_TRAINING_CONFIG["gradient_accumulation_steps"]

for epoch in range(REWARD_TRAINING_CONFIG["num_train_epochs"]):
    print(f"\n📊 Epoch {epoch + 1}/{REWARD_TRAINING_CONFIG['num_train_epochs']}")
    
    total_loss = 0
    optimizer.zero_grad()
    
    progress_bar = tqdm(train_dataloader, desc=f"Training")
    
    for step, batch in enumerate(progress_bar):
        # Tokenize chosen
        chosen_inputs = tokenizer(
            batch['chosen'],
            return_tensors='pt',
            padding=True,
            truncation=True,
            max_length=1024
        )
        
        # Tokenize rejected
        rejected_inputs = tokenizer(
            batch['rejected'],
            return_tensors='pt',
            padding=True,
            truncation=True,
            max_length=1024
        )
        
        # Move to device
        chosen_inputs = {k: v.to(device) for k, v in chosen_inputs.items()}
        rejected_inputs = {k: v.to(device) for k, v in rejected_inputs.items()}
        
        # Forward pass
        chosen_rewards = reward_model(**chosen_inputs).logits
        rejected_rewards = reward_model(**rejected_inputs).logits
        
        # Loss: chosen should have higher reward
        # Using ranking loss: -log(sigmoid(chosen - rejected))
        loss = -torch.nn.functional.logsigmoid(chosen_rewards - rejected_rewards).mean()
        
        # Scale loss for gradient accumulation
        loss = loss / gradient_accumulation_steps
        
        # Backward
        loss.backward()
        
        total_loss += loss.item()
        
        # Update weights every N steps
        if (step + 1) % gradient_accumulation_steps == 0:
            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(reward_model.parameters(), 1.0)
            
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
        
        # Update progress bar
        progress_bar.set_postfix({
            'loss': f'{loss.item() * gradient_accumulation_steps:.4f}',
            'lr': f'{scheduler.get_last_lr()[0]:.2e}'
        })
        
        # Logging
        if (step + 1) % REWARD_TRAINING_CONFIG["logging_steps"] == 0:
            avg_loss = (total_loss * gradient_accumulation_steps) / (step + 1)
            print(f"  Step {step + 1}: loss={avg_loss:.4f}")
    
    # Epoch summary
    avg_loss = (total_loss * gradient_accumulation_steps) / len(train_dataloader)
    print(f"\n✅ Epoch {epoch + 1} Complete - Average Loss: {avg_loss:.4f}")
    
    # Evaluation
    if eval_dataloader and len(eval_dataloader) > 0:
        print(f"\n📊 Running evaluation...")
        reward_model.eval()
        
        eval_loss = 0
        eval_steps = 0
        
        with torch.no_grad():
            for batch in tqdm(eval_dataloader, desc="Evaluating"):
                chosen_inputs = tokenizer(
                    batch['chosen'],
                    return_tensors='pt',
                    padding=True,
                    truncation=True,
                    max_length=1024
                )
                
                rejected_inputs = tokenizer(
                    batch['rejected'],
                    return_tensors='pt',
                    padding=True,
                    truncation=True,
                    max_length=1024
                )
                
                chosen_inputs = {k: v.to(device) for k, v in chosen_inputs.items()}
                rejected_inputs = {k: v.to(device) for k, v in rejected_inputs.items()}
                
                chosen_rewards = reward_model(**chosen_inputs).logits
                rejected_rewards = reward_model(**rejected_inputs).logits
                
                loss = -torch.nn.functional.logsigmoid(chosen_rewards - rejected_rewards).mean()
                
                eval_loss += loss.item()
                eval_steps += 1
        
        avg_eval_loss = eval_loss / eval_steps
        print(f"✅ Evaluation Loss: {avg_eval_loss:.4f}")
        
        reward_model.train()


STEP 1: TRAINING REWARD MODEL
Loading base model for reward modeling...


config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

...

  Step 890: loss=1.0357


Training: 100%|██████████| 900/900 [1:08:55<00:00,  4.60s/it, loss=0.7497, lr=6.22e-06]


  Step 900: loss=1.0328

✅ Epoch 1 Complete - Average Loss: 1.0328

📊 Running evaluation...


Evaluating: 100%|██████████| 100/100 [02:39<00:00,  1.59s/it]

✅ Evaluation Loss: 0.8977


In [24]:
# Save reward model
REWARD_MODEL_PATH= "/kaggle/working/reward_model_final"

print(f"💾 Saving reward model to {REWARD_MODEL_PATH}...")
reward_model.save_pretrained(REWARD_MODEL_PATH)
tokenizer.save_pretrained(REWARD_MODEL_PATH)

print("✅ Reward model training complete!")

💾 Saving reward model to /kaggle/working/reward_model_final...
✅ Reward model training complete!


In [25]:
import shutil
import os

REWARD_MODEL_PATH = "/kaggle/working/reward_model_final"
ZIP_PATH = "/kaggle/working/reward_model_final.zip"

# Remove zip if it already exists
if os.path.exists(ZIP_PATH):
    os.remove(ZIP_PATH)

# Create zip
shutil.make_archive(
    base_name=ZIP_PATH.replace(".zip", ""),
    format="zip",
    root_dir=REWARD_MODEL_PATH
)

print(f"📦 Zipped reward model saved to {ZIP_PATH}")


📦 Zipped reward model saved to /kaggle/working/reward_model_final.zip


## Load Model_SFT for PPO

In [16]:
print("\n" + "="*80)
print("STEP 2: PREPARING FOR PPO TRAINING")
print("="*80)

# Load Model_SFT (your baseline from Step 1)
print("Loading Model_SFT...")
base_model_ppo = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=BNB_CONFIG,
    device_map="auto",
    trust_remote_code=True,
)

# Load LoRA adapter (Model_SFT)
model_sft = PeftModel.from_pretrained(base_model_ppo, MODEL_SFT_PATH)

# Wrap model for PPO (adds value head)
ppo_model = AutoModelForCausalLMWithValueHead.from_pretrained(model_sft)

print("✅ Model_SFT loaded and wrapped for PPO!")


STEP 2: PREPARING FOR PPO TRAINING
Loading Model_SFT...


config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

✅ Model_SFT loaded and wrapped for PPO!


/usr/local/lib/python3.12/dist-packages/trl/experimental/ppo/modeling_value_head.py:273: FutureWarning: The `AutoModelForCausalLMWithValueHead` is now located in `trl.experimental`. Please update your imports to `from trl.experimental.ppo import AutoModelForCausalLMWithValueHead`. The current import path will be removed and no longer supported in TRL 0.29. For more information, see https://github.com/huggingface/trl/issues/4223.
  model = cls(pretrained_model, **multi_adapter_args, **trl_model_args)


## Load Reward Model for Inference

In [17]:
print("Loading trained reward model...")

# Load the trained reward model
reward_model_inference = AutoModelForSequenceClassification.from_pretrained(
    BASE_MODEL,
    num_labels=1,
    quantization_config=BNB_CONFIG,
    device_map="auto",
    trust_remote_code=True,
)

# Load LoRA weights
reward_model_inference = PeftModel.from_pretrained(
    reward_model_inference,
    REWARD_MODEL_PATH
)
reward_model_inference.eval()

print("✅ Reward model loaded for inference!")

Loading trained reward model...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-3.2-3B-Instruct and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ Reward model loaded for inference!


## Prepare PPO Dataset

In [18]:
def prepare_ppo_dataset(dataset: Dataset) -> Dataset:
    """Prepare dataset for PPO training (only prompts needed)"""
    
    def format_prompt(example):
        prompt = example['prompt']
        formatted = f"<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\n{prompt}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"
        return {'query': formatted}
    
    return dataset.map(format_prompt, remove_columns=dataset.column_names)

ppo_train_dataset = prepare_ppo_dataset(train_dataset)

print(f"✅ PPO dataset prepared: {len(ppo_train_dataset)} prompts")
print(f"Sample query: {ppo_train_dataset[0]['query'][:100]}...")

Map:   0%|          | 0/900 [00:00<?, ? examples/s]

✅ PPO dataset prepared: 900 prompts
Sample query: <|begin_of_text|><|start_header_id|>user<|end_header_id|>

A 25 yr old female presented with malaise...


## PPO Training Loop

In [19]:
import torch
import numpy as np
from tqdm import tqdm
from torch.utils.data import DataLoader
import copy
import shutil
from typing import List
from transformers import GenerationConfig
from trl.experimental.ppo import PPOConfig, PPOTrainer

print("\n" + "="*80)
print("STEP 3: PPO TRAINING (Experimental API)")
print("="*80)

# =========================
# PPO CONFIG
# =========================
ppo_config = PPOConfig(**PPO_CONFIG)

# =========================
# POLICY MODEL SETUP
# =========================
policy_model = ppo_model.half().cuda() 

# Assign generation_config to policy model
policy_model.generation_config = GenerationConfig(
    max_new_tokens=256,
    temperature=0.7,
    top_p=0.9,
    do_sample=True,
    pad_token_id=tokenizer.pad_token_id,
    eos_token_id=tokenizer.eos_token_id,
)

# =========================
# VALUE MODEL (frozen copy)
# =========================
print("Creating value model...")
value_model = copy.deepcopy(policy_model).eval().cpu() 
for p in value_model.parameters():
    p.requires_grad = False
print("✅ Value model created on CPU")

# =========================
# REFERENCE MODEL (frozen)
# =========================
print("Creating frozen reference model...")
ref_model = copy.deepcopy(policy_model.pretrained_model).eval().cpu() 
for p in ref_model.parameters():
    p.requires_grad = False
print("✅ Reference model created on CPU")

# =========================
# REWARD MODEL ON CPU
# =========================
reward_model_inference = reward_model_inference.half().cpu()
print("✅ Reward model on CPU")

# =========================
# REWARD FUNCTION WITH CPU OFFLOAD
# =========================
def get_reward(response_texts: List[str], prompt_texts: List[str]) -> List[float]:
    """Compute rewards for generated responses"""
    texts = []
    for prompt, response in zip(prompt_texts, response_texts):
        text = f"<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\n{prompt}<|eot_id|>"
        text += f"<|start_header_id|>assistant<|end_header_id|>\n\n{response}<|eot_id|>"
        texts.append(text)

    inputs = tokenizer(
        texts,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=256,
    )

    device = torch.device("cuda")
    reward_model_inference.to(device)
    
    # Keep input_ids as long type, don't convert to half
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.autocast("cuda"), torch.no_grad():
        rewards = reward_model_inference(**inputs).logits.squeeze(-1)

    reward_model_inference.cpu()
    torch.cuda.empty_cache()
    
    return rewards.cpu().tolist()

# =========================
# PPO TRAINER SETUP
# =========================
print("\nCreating PPO trainer...")
ppo_trainer = PPOTrainer(
    ppo_config,
    processing_class=tokenizer,
    model=policy_model.pretrained_model,
    ref_model=ref_model,
    reward_model=reward_model_inference,
    value_model=value_model.pretrained_model,
    train_dataset=ppo_train_dataset,
)

print("✅ PPO trainer created!")

# =========================
# OPTION 1: USE BUILT-IN TRAIN() METHOD
# =========================
print("\n🚀 Starting PPO training using trainer.train()...")
print("Note: The trainer will handle the training loop internally\n")

try:
    # Use the built-in train() method
    # This should handle generation, reward computation, and PPO updates automatically
    ppo_trainer.train()
    
    print("\n✅ Training completed successfully!")
    
except Exception as e:
    print(f"\n⚠️ Built-in train() failed: {str(e)}")
    print("Falling back to manual training loop...\n")
    import traceback
    traceback.print_exc()
    
    # =========================
    # OPTION 2: MANUAL TRAINING LOOP
    # =========================
    print("\n" + "="*80)
    print("Using manual training loop with training_step()")
    print("="*80)
    
    # Create dataloader
    def collate_fn(batch):
        """Collate function that tokenizes text queries"""
        query_texts = [item['query'] for item in batch]
        tokenized = tokenizer(
            query_texts,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=512,
        )
        return tokenized

    ppo_dataloader = DataLoader(
        ppo_train_dataset,
        batch_size=1, 
        shuffle=True,
        drop_last=True,
        collate_fn=collate_fn,
    )
    
    generation_kwargs = {
        "max_new_tokens": 256,
        "temperature": 0.7,
        "top_p": 0.9,
        "do_sample": True,
        "pad_token_id": tokenizer.pad_token_id,
        "eos_token_id": tokenizer.eos_token_id,
    }
    
    for epoch in range(1):
        print(f"\n{'='*80}")
        print(f"📊 EPOCH {epoch+1}/1")
        print(f"{'='*80}")

        epoch_rewards = []
        
        for batch_idx, batch in enumerate(tqdm(ppo_dataloader, desc=f"Epoch {epoch+1}")):
            # Prepare inputs
            query_tensors = batch["input_ids"].cuda()
            attention_mask = batch["attention_mask"].cuda()
            
            # Generate responses
            with torch.autocast("cuda"), torch.no_grad():
                response_tensors = policy_model.generate(
                    query_tensors,
                    attention_mask=attention_mask,
                    **generation_kwargs
                )

            # Decode queries
            queries_decoded = []
            for q in query_tensors:
                decoded = tokenizer.decode(q, skip_special_tokens=True)
                if "user<|end_header_id|>" in decoded:
                    prompt = decoded.split("user<|end_header_id|>")[1].split("<|eot_id|>")[0].strip()
                else:
                    prompt = decoded
                queries_decoded.append(prompt)

            # Decode responses
            responses_decoded = []
            for r in response_tensors:
                decoded = tokenizer.decode(r, skip_special_tokens=True)
                if "assistant<|end_header_id|>" in decoded:
                    response = decoded.split("assistant<|end_header_id|>")[-1].split("<|eot_id|>")[0].strip()
                else:
                    response = decoded
                responses_decoded.append(response)

            # Get rewards
            rewards = get_reward(responses_decoded, queries_decoded)
            rewards_tensor = torch.tensor(rewards, device=query_tensors.device, dtype=torch.float16)

            # Create batch dict for training_step
            batch_dict = {
                'input_ids': query_tensors,
                'attention_mask': attention_mask,
                'response_ids': response_tensors,
                'rewards': rewards_tensor,
            }
            
            try:
                # Use training_step with the batch
                loss = ppo_trainer.training_step(policy_model.pretrained_model, batch_dict)
                
                epoch_rewards.extend(rewards)
                
                if batch_idx % 10 == 0:
                    print(f"\n  Batch {batch_idx}: Reward: {np.mean(rewards):.3f}, Loss: {loss:.4f}")
                    if batch_idx == 0:
                        print(f"  Sample prompt: {queries_decoded[0][:100]}...")
                        print(f"  Sample response: {responses_decoded[0][:100]}...")
            
            except Exception as e:
                print(f"\n⚠️ Error in batch {batch_idx}: {str(e)}")
                import traceback
                traceback.print_exc()
                continue

            if batch_idx % 50 == 0:
                torch.cuda.empty_cache()

        # Print statistics
        if epoch_rewards:
            print(f"\n{'='*80}")
            print(f"📈 EPOCH {epoch+1} COMPLETE")
            print(f"   Mean Reward: {np.mean(epoch_rewards):.3f}")
            print(f"   Std Reward: {np.std(epoch_rewards):.3f}")
            print(f"   Min Reward: {np.min(epoch_rewards):.3f}")
            print(f"   Max Reward: {np.max(epoch_rewards):.3f}")
            print(f"{'='*80}\n")

# =========================
# SAVE CHECKPOINT
# =========================
checkpoint_path = f"/kaggle/working/ppo_final"
print(f"\n💾 Saving final checkpoint to {checkpoint_path}...")
policy_model.save_pretrained(checkpoint_path)
tokenizer.save_pretrained(checkpoint_path)
shutil.make_archive(checkpoint_path, 'zip', checkpoint_path)
print(f"✅ Checkpoint saved and zipped!")

print("\n" + "="*80)
print("🎉 PPO TRAINING COMPLETE!")
print("="*80)


STEP 3: PPO TRAINING (Experimental API)
Creating value model...
✅ Value model created on CPU
Creating frozen reference model...
✅ Reference model created on CPU
✅ Reward model on CPU

Creating PPO trainer...
✅ PPO trainer created!

🚀 Starting PPO training using trainer.train()...
Note: The trainer will handle the training loop internally

===training policy===


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

KeyboardInterrupt: 

In [32]:
"""
PPOTrainer API Diagnostic Script
Run this to identify the correct method to use with the experimental PPOTrainer
"""

print("\n" + "="*80)
print("PPO TRAINER API DIAGNOSTIC")
print("="*80)

print("\n1. All public methods and attributes:")
print("-" * 80)
for attr in sorted(dir(ppo_trainer)):
    if not attr.startswith('_'):
        obj = getattr(ppo_trainer, attr)
        if callable(obj):
            print(f"  ✓ {attr}() - method")
        else:
            print(f"  • {attr} - attribute")

print("\n2. Looking for training-related methods:")
print("-" * 80)
training_keywords = ['step', 'update', 'train', 'forward', 'batch', 'loss', 'optimize']
found_methods = []
for attr in dir(ppo_trainer):
    if not attr.startswith('_'):
        if any(keyword in attr.lower() for keyword in training_keywords):
            obj = getattr(ppo_trainer, attr)
            if callable(obj):
                found_methods.append(attr)
                print(f"  → {attr}()")

if found_methods:
    print(f"\n3. Most likely training methods: {', '.join(found_methods)}")
else:
    print("\n3. ⚠️ No obvious training methods found")

print("\n4. Checking method signatures:")
print("-" * 80)
import inspect
for method_name in found_methods[:5]:  # Check first 5 methods
    try:
        method = getattr(ppo_trainer, method_name)
        sig = inspect.signature(method)
        print(f"\n{method_name}{sig}")
        # Try to get docstring
        if method.__doc__:
            doc_lines = method.__doc__.strip().split('\n')[:3]
            for line in doc_lines:
                print(f"    {line.strip()}")
    except Exception as e:
        print(f"\n{method_name}: Could not inspect - {e}")

print("\n" + "="*80)
print("DIAGNOSTIC COMPLETE")
print("="*80)

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.



PPO TRAINER API DIAGNOSTIC

1. All public methods and attributes:
--------------------------------------------------------------------------------
  • accelerator - attribute
  ✓ add_callback() - method
  • args - attribute
  ✓ autocast_smart_context_manager() - method
  ✓ call_model_init() - method
  • callback_handler - attribute
  • callbacks - attribute
  ✓ compare_trainer_and_checkpoint_args() - method
  ✓ compute_loss() - method
  ✓ compute_loss_context_manager() - method
  • control - attribute
  ✓ create_accelerator_and_postprocess() - method
  ✓ create_model_card() - method
  ✓ create_optimizer() - method
  ✓ create_optimizer_and_scheduler() - method
  ✓ create_scheduler() - method
  • current_flos - attribute
  ✓ data_collator() - method
  • dataloader - attribute
  • eval_dataloader - attribute
  • eval_dataset - attribute
  ✓ evaluate() - method
  ✓ evaluation_loop() - method
  ✓ floating_point_ops() - method
  ✓ generate_completions() - method
  ✓ get_batch_samples() - me

## Save Final Model C

In [ ]:
# =========================
# SAVE FINAL MODEL
# =========================
final_model_path = "/kaggle/working/model_c_ppo_final"
policy_model.save_pretrained(final_model_path)
tokenizer.save_pretrained(final_model_path)
shutil.make_archive(final_model_path, 'zip', final_model_path)

print("\n✅ PPO TRAINING COMPLETE (1 EPOCH)!")

## Test Model C

In [ ]:
print("\n" + "="*80)
print("TESTING MODEL C")
print("="*80)

# Test prompt
test_prompt = "A 55-year-old man presents with chest pain and shortness of breath. What are the key differential diagnoses?"

formatted_prompt = f"<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\n{test_prompt}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"

# Generate
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(ppo_model.device)

with torch.no_grad():
    outputs = ppo_model.generate(
        **inputs,
        max_new_tokens=256,
        temperature=0.7,
        top_p=0.9,
    )

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"\n📝 Prompt: {test_prompt}")
print(f"\n🤖 Response:\n{response}")

print("\n" + "="*80)
print("ALL DONE! 🎉")
print("="*80)
print(f"\nModel C (RLHF + PPO) is ready at: {MODEL_C_PATH}")
print("\nNext steps:")
print("1. Download Model C from Kaggle")
print("2. Compare with Model A, B, and D")
print("3. Evaluate on test set")